# **Costa Rica - Seizure and Homicide Data Analysis**
#### InSight Crime - MAD Unit 
June, 2025


---------------------



### Introduction

Data:

1. **Intentional Homicides in Costa Rica (2010–2024)**
     
	Formatted data available [here](https://docs.google.com/spreadsheets/d/1-XG8sMyPwBXa6bdBCqqEPm2SrcHpKKn3YAHaBdyR-Lg/edit?gid=0#gid=0)  
	Source: [Intentional Homicides CR](https://drive.google.com/file/d/1cUswepSw-r3HLU6Be3duvjsRw329MoKg/view?ts=6841b341)
	
	Notes: There are some discrepancies in the population figures by year compared to the [INEC](https://services.inec.go.cr/proyeccionpoblacion/frmproyec.aspx) (at both national and provincial levels), but they do not affect the homicide rates per 100,000 inhabitants as reported in the source.

2. **Seizure data (2018-2024)**
   
	Source: [Control_de_la_Oferta_ODS_SW1 (Sheet "8")](https://docs.google.com/spreadsheets/d/1EhZ7hpFBpIFc375ByzMu1AIQrNnvw2FxIvXlxV5KE1E/edit?gid=661345260#gid=661345260) 

3. **Merged dataset**

	Available [here](https://github.com/FelipeVillota/costa-rica-presentation/tree/main/data/merged) 
	
	Notes: Dataframes were combined considering geographic disaggregation and time. Leaving us with country-level observations/indicators, by year, from 2018-2014. This is the starting point for the analysis


### Project Setup

#### Version Control

The project is created within a single GitHub repository ([FelipeVillota/costa-rica-presentation](https://github.com/FelipeVillota/costa-rica-presentation)). I keep the repository `private` with the possibility to give collaborator-access to the online repo at any time. 

#### Reproducible Environment

In [65]:
# IMPORTANT

# In the Terminal, run the following commands to set up a virtual environment called `venv-cr`:
# python -m venv venv-cr

# To activate environment, run (the first is an optional, temporary auth) :
# Set-ExecutionPolicy -Scope Process -ExecutionPolicy Bypass
# venv-cr\Scripts\activate

# Then select respective kernel 

# Update list master list
# pip freeze > requirements.txt

In [66]:
# Checking venv-db-watch works --> accept to install ipykernel package to connect to kernel if asked
import sys
print(sys.executable)

c:\Users\USER\Desktop\ic\costa-rica\costa-rica-presentation\venv-cr\Scripts\python.exe


#### Loading Libraries

In [67]:
# Install the required packages in the virtual environment:
# pip install --upgrade pip
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib pandas
# pip install gspread gspread-formatting seaborn matplotlib missingno    

In [68]:
import os
import re
import requests
import pandas as pd
from datetime import datetime
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread
from google.oauth2.service_account import Credentials
from gspread_formatting import format_cell_ranges, CellFormat, Color
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from io import BytesIO
from openpyxl import load_workbook


# pip freeze > requirements.txt

### Loading Data

In [69]:
# Both datasets are in the repository, so we can use raw GitHub URLs to access them directly.
# Use raw GitHub URLs
url_homicides = "https://github.com/FelipeVillota/costa-rica-presentation/raw/main/data/raw/homicides-2010-2024.xlsx" # Multiple sheets 
url_seizures = "https://github.com/FelipeVillota/costa-rica-presentation/raw/main/data/raw/seizures-2019-2024.xlsx" # This is already the sheet we want

# Get file content
response1 = requests.get(url_homicides)
response2 = requests.get(url_seizures)

In [70]:
# Seizures data is already in a single sheet, so we can load it directly
seizures_sheets = pd.read_excel(BytesIO(response2.content), engine='openpyxl', sheet_name=None)
seizures_df = seizures_sheets['8']  


In [71]:
# Homicide data is in multiple sheets, we want to load only the second sheet called "tasa-país"
homicides_sheets = pd.read_excel(BytesIO(response1.content), engine='openpyxl', sheet_name=None)
homicides_df = homicides_sheets['tasa-país']

In [72]:
# All sheets
homicides_sheets


{'histórico-mensual':            país         mes   año  homicidios_país_mes  \
 0    Costa Rica       enero  2010                   44   
 1    Costa Rica     febrero  2010                   45   
 2    Costa Rica       marzo  2010                   47   
 3    Costa Rica       abril  2010                   42   
 4    Costa Rica        mayo  2010                   39   
 ..          ...         ...   ...                  ...   
 175  Costa Rica      agosto  2024                   75   
 176  Costa Rica  septiembre  2024                   69   
 177  Costa Rica     octubre  2024                   73   
 178  Costa Rica   noviembre  2024                   73   
 179  Costa Rica   diciembre  2024                   80   
 
      homicidios_país_acumulado_mes  homicidios_país_año  
 0                              756                  527  
 1                              690                  527  
 2                              699                  527  
 3                              6

In [73]:
print(seizures_df.head())


         País Provincia    Cantón     Distrito   Cocaína Eventos  \
0  Costa Rica  Alajuela  Alajuela     Alajuela             161.0   
1  Costa Rica  Alajuela  Alajuela     San José             144.0   
2  Costa Rica  Alajuela  Alajuela     Carrizal               9.0   
3  Costa Rica  Alajuela  Alajuela  San Antonio              34.0   
4  Costa Rica  Alajuela  Alajuela      Guácima              51.0   

   Cocaína (kg)  Crack Eventos  Crack (piedras)  Marihuana Eventos  \
0       0.05825          370.0      6644.400000              769.0   
1       0.70250          289.0     10406.666667              478.0   
2       0.00625            9.0       204.000000               21.0   
3       0.32975           60.0       913.333333              449.0   
4       0.93440          155.0      5812.333333              450.0   

   Marihuana (kg)     Año  
0        14.33302  2019.0  
1         1.45790  2019.0  
2         0.03950  2019.0  
3         1.06203  2019.0  
4         2.08663  2019.0  


In [74]:
# Now we want to merge the two datasets on the year column.
 
# But first, we need to ensure that the year columns in both datasets are in the same format.
homicides_df['año'] = pd.to_datetime(homicides_df['año'], errors='coerce', format='%Y').dt.year
seizures_df['Año'] = pd.to_datetime(seizures_df['Año'], format='%Y').dt.year

# And change the column name to match
homicides_df.rename(columns={'año': 'year'}, inplace=True)
seizures_df.rename(columns={'Año': 'year'}, inplace=True)


In [75]:
# seizure data is showing values like 2019.0, so we need to convert it to int
seizures_df['year'] = seizures_df['year'].astype('Int64')

In [76]:
# Now we can merge the two datasets on the 'year' column.
merged_df = pd.merge(homicides_df, seizures_df, on='year', how='outer')

In [77]:
# To keep it simple, we keep only record from 2018 to 2024
merged_df = merged_df[merged_df['year'] >= 2018]

In [91]:
merged_df.head()
print(merged_df.info())

# Now we can save the merged dataset to a new Excel file.
def save_merged_df(merged_df, output_path='data/merged/merged_data-2018-2024.xlsx'):
    merged_df.to_excel(output_path, index=False)
save_merged_df(merged_df)

<class 'pandas.core.frame.DataFrame'>
Index: 3398 entries, 8 to 3405
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   país                 3398 non-null   object 
 1   provincia            3398 non-null   object 
 2   cantón               3398 non-null   object 
 3   distrito             3398 non-null   object 
 4   year                 3398 non-null   Int64  
 5   habitantes_país      3398 non-null   float64
 6   homicidios_país_año  3398 non-null   float64
 7   tasa_100mil_país     3398 non-null   float64
 8   cocaína_eventos      3398 non-null   float64
 9   cocaína_(kg)         3398 non-null   float64
 10  crack_eventos        3398 non-null   float64
 11  crack_(piedras)      3398 non-null   float64
 12  marihuana_eventos    3398 non-null   float64
 13  marihuana_(kg)       3398 non-null   float64
dtypes: Int64(1), float64(9), object(4)
memory usage: 401.5+ KB
None


### Execution

#### Pre-processing

In [79]:
# Format Missing/Invalid Values

merged_df = merged_df.replace('', pd.NA)  # Convert empty strings to NA
merged_df = merged_df.replace(r'^\s*$', pd.NA, regex=True)  # Convert whitespace to NA
print("✓ Converted empty strings/whitespace to NA values")

✓ Converted empty strings/whitespace to NA values


In [80]:
# Lowercase column names and put _ between words
merged_df.columns = [col.lower() for col in merged_df.columns]

# Remove spaces left and right
merged_df.columns = merged_df.columns.str.strip()

#  put _ between words
merged_df.columns = [re.sub(r'\s+', '_', col.lower()) for col in merged_df.columns]

print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3398 entries, 8 to 3405
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   país                 3398 non-null   object 
 1   year                 3398 non-null   Int64  
 2   homicidios_país_año  3398 non-null   float64
 3   habitantes_país      3398 non-null   float64
 4   tasa_100mil_país     3398 non-null   float64
 5   país                 3398 non-null   object 
 6   provincia            3398 non-null   object 
 7   cantón               3398 non-null   object 
 8   distrito             3398 non-null   object 
 9   cocaína_eventos      3398 non-null   float64
 10  cocaína_(kg)         3398 non-null   float64
 11  crack_eventos        3398 non-null   float64
 12  crack_(piedras)      3398 non-null   float64
 13  marihuana_eventos    3398 non-null   float64
 14  marihuana_(kg)       3398 non-null   float64
dtypes: Int64(1), float64(9), object(5)
memory u

In [88]:
# Drop the second (duplicated) 'país' column
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

# Arrange columns in a more logical order 

column_order = [
    'país', 'provincia', 'cantón', 'distrito', 'year',
    'habitantes_país', 'homicidios_país_año', 'tasa_100mil_país',
    'cocaína_eventos', 'cocaína_(kg)',
    'crack_eventos', 'crack_(piedras)',
    'marihuana_eventos', 'marihuana_(kg)'
]
merged_df = merged_df[column_order]

### Exploratory Data Analysis

##### Summary Stats

In [89]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

def get_dataframe_summary(merged_df, output_path='data/results/merged_summary-2018-2024.xlsx'):
    # --- General DataFrame Info ---
    general_info = {
        "Shape": f"{merged_df.shape[0]} rows × {merged_df.shape[1]} cols",
        "Memory Usage": f"{merged_df.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB",
        "Columns with NA": f"{merged_df.isna().any().sum()} / {len(merged_df.columns)}",
        "Duplicate Rows": f"{merged_df.duplicated().sum()} ({(merged_df.duplicated().mean() * 100):.1f}%)",
        "Numeric Columns": f"{merged_df.select_dtypes(include='number').shape[1]}",
        "Categorical Columns": f"{merged_df.select_dtypes(include=['object', 'category']).shape[1]}",
        "Datetime Columns": f"{merged_df.select_dtypes(include='datetime').shape[1]}"
    }

    # --- Column-Level Stats ---
    column_stats = pd.DataFrame({
        'Variable': merged_df.columns,
        'Dtype': merged_df.dtypes.values,
        'Unique_Count': merged_df.nunique().values,
        'NA_Count': merged_df.isna().sum().values,
        'NA_Percentage': (merged_df.isna().mean() * 100).round(1).values,
        'Duplicate_Count': merged_df.apply(lambda col: col.duplicated(keep=False).sum()).values,
        'Duplicate_Percentage': (merged_df.apply(lambda col: col.duplicated(keep=False).mean()) * 100).round(1).values,
        'Unique_Values': merged_df.apply(lambda x: x.drop_duplicates().tolist()).values
    }).sort_values('Unique_Count', ascending=False)


    # Format percentages
    column_stats['NA_Percentage'] = column_stats['NA_Percentage'].astype(str) + '%'
    column_stats['Duplicate_Percentage'] = column_stats['Duplicate_Percentage'].astype(str) + '%'
    
# --- Save to Excel ---
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        # Save general info to first sheet
        pd.DataFrame.from_dict(general_info, orient='index', columns=['Value'])\
            .to_excel(writer, sheet_name='General Info')
        
        # Save column stats to second sheet
        column_stats.to_excel(writer, sheet_name='Column Stats', index=False)
        
        # Auto-adjust column widths
        for sheet_name in writer.sheets:
            sheet = writer.sheets[sheet_name]
            for column in sheet.columns:
                max_length = 0
                column_letter = column[0].column_letter
                for cell in column:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(str(cell.value))
                    except:
                        pass
                adjusted_width = (max_length + 2)
                sheet.column_dimensions[column_letter].width = adjusted_width

    print(f"✓ Summary saved to {output_path}")
    return general_info, column_stats

# Run the summary
general_info, column_stats = get_dataframe_summary(merged_df)

# Print General Info
print("=== GENERAL DATAFRAME INFO ===")
for key, value in general_info.items():
    print(f"{key}: {value}")

# Display Column Stats (including dropdown options)
print("\n=== COLUMN-LEVEL STATISTICS ===")
display(column_stats)


✓ Summary saved to data/results/merged_summary-2018-2024.xlsx
=== GENERAL DATAFRAME INFO ===
Shape: 3398 rows × 14 cols
Memory Usage: 1.11 MB
Columns with NA: 0 / 14
Duplicate Rows: 0 (0.0%)
Numeric Columns: 10
Categorical Columns: 4
Datetime Columns: 0

=== COLUMN-LEVEL STATISTICS ===


,Variable,Dtype,Unique_Count,NA_Count,NA_Percentage,Duplicate_Count,Duplicate_Percentage,Unique_Values
13,marihuana_(kg),float64,2651,0,0.0%,976,28.7%,"[3.79137949254408, 8.64565, 0.13497, 1.28543, ..."
11,crack_(piedras),float64,1511,0,0.0%,2217,65.2%,"[9051.13333333333, 17697.2222222222, 87.0, 780..."
9,cocaína_(kg),float64,1287,0,0.0%,2316,68.2%,"[0.1469, 0.23475, 0.0015, 0.1175, 0.03345, 0.8..."
12,marihuana_eventos,float64,632,0,0.0%,3120,91.8%,"[842.0, 249.0, 55.0, 336.0, 219.0, 27.0, 77.0,..."
3,distrito,object,397,0,0.0%,3395,99.9%,"[Alajuela, San José, Carrizal, San Antonio, Gu..."
10,crack_eventos,float64,305,0,0.0%,3256,95.8%,"[350.0, 164.0, 15.0, 66.0, 55.0, 6.0, 7.0, 146..."
8,cocaína_eventos,float64,211,0,0.0%,3308,97.4%,"[100.0, 74.0, 3.0, 39.0, 21.0, 5.0, 54.0, 78.0..."
2,cantón,object,93,0,0.0%,3396,99.9%,"[Alajuela, San Ramón, Grecia, San Mateo, Atena..."
1,provincia,object,12,0,0.0%,3397,100.0%,"[Alajuela, Cartago, Guanacaste, Heredia, Limón..."
4,year,Int64,7,0,0.0%,3398,100.0%,"[2018, 2019, 2020, 2021, 2022, 2023, 2024]"
